# 사용할 모듈 불러오기

In [150]:
import pandas as pd
import requests
from bs4 import BeautifulSoup as bs
import time
import numpy as np

# URL 지정

In [ ]:
page_no = 1
url = f"https://finance.naver.com/item/board.nhn?code=005930&page={page_no}"

# requests를 통해 http 요청

In [67]:
headers = {'user-agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.150 Safari/537.36'}
response = requests.get(url, headers = headers)
response.text

'<script type="text/javascript">\nfunction doSearch() {\n\tvar frm = document.boardSearchFrm;\n\tvar str = frm.sw.value;\n\tstr = str.replace(/^\\s*/,\'\').replace(/\\s*$/,\'\');\t// trim\n\t\n\tif(str == \'\') {\n\t\talert("검색어를 입력해 주세요.");\n\t\tfrm.sw.value = \'\';\n\t\tfrm.sw.focus();\n\t}\n\telse {\n\t\tfrm.sw.value = str;\n\t\tfrm.submit();\n\t}\n}\n</script>\n\n<!--  global include -->\n\n\t\n\n<html lang=\'ko\'>\n<head>\n\n\n\t\n\t\t<title>삼성전자 : 네이버 금융</title>\n\t\n\t\n\n\n\n\n\n\t\n\t\n\t\t<meta http-equiv="Content-Type" content="text/html; charset=utf-8" />\n\t\n\n\n<meta http-equiv="Content-Script-Type" content="text/javascript">\n<meta http-equiv="Content-Style-Type" content="text/css">\n\n\n\n\n\n\t\n\t\t<meta property="og:title" content="네이버 금융"/>\n\t\t<meta property="og:image" content="https://ssl.pstatic.net/static/m/stock/im/2016/08/og_stock-200.png"/>\n\t\t<meta property="og:url" content="https://finance.naver.com"/>\n\t\t<meta property="og:description" content="국내 해외

# BeautifulSoup을 통해 table tag 및 마지막 페이지 번호 찾기

## table tag

In [68]:
html = bs(response.text)

In [69]:
tables = html.select("table")
table = pd.read_html(str(tables), encoding = "euc-kr")

In [77]:
data = table[1].iloc[:,:6].dropna()
data.head(3)

,날짜,제목,글쓴이,조회,공감,비공감
1,2021.02.16 17:47,46800에 매수한 120살 할머니입니다,aass****,5.0,0.0,0.0
2,2021.02.16 17:46,오빠들 저 어떡해요 ㅜㅜ 인생 망했어요....,rlfd****,40.0,0.0,1.0
3,2021.02.16 17:43,가장 드러운 장세다..위에서 계속 누른다...,saba****,46.0,2.0,1.0


## 마지막 페이지 번호

In [118]:
html.select(".pgRR > a")[0]["href"].split("=")[-1]

'46913'

# 함수 생성

## 페이지별 데이터 수집 함수 생성

In [81]:

def get_page_data(item_code, page_no, headers = None):
    """
    특정 주식 종목 및 종목토론실 글을 불러오기
    """
    url = f"https://finance.naver.com/item/board.nhn?code={item_code}&page={page_no}"
    response = requests.get(url,headers = headers)
    
    html = bs(response.text)
    
    tables = html.select("table")
    table = pd.read_html(str(tables),encoding = "euc-kr")
    
    data = table[1].iloc[:,:6].dropna()
    return data

In [82]:
item_code = "005930"
page_no = 1
headers = {'user-agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.150 Safari/537.36'}
get_page_data(item_code = item_code, page_no = page_no, headers = headers)

,날짜,제목,글쓴이,조회,공감,비공감
1,2021.02.16 17:54,봐봐 정치라는거 어떤것인가 가장 지혜로운...,true****,2.0,0.0,0.0
2,2021.02.16 17:53,왜 우리나라 100신은 칼라냐 흰색 검은...,kshe****,5.0,0.0,0.0
3,2021.02.16 17:51,계속 사야합니다,heis****,37.0,0.0,0.0
4,2021.02.16 17:48,단타를쳐야지 글게 쳐물려,gmdq****,92.0,1.0,3.0
5,2021.02.16 17:47,삼성 목표주가 15만원 [1],rys0****,210.0,7.0,3.0
7,2021.02.16 17:47,46800에 매수한 120살 할머니입니다 [1],aass****,151.0,3.0,2.0
8,2021.02.16 17:46,오빠들 저 어떡해요 ㅜㅜ 인생 망했어요.... [5],rlfd****,328.0,1.0,9.0
9,2021.02.16 17:43,초고수다..가장 드러운 장세다..위에서 ... [1],saba****,158.0,2.0,2.0
10,2021.02.16 17:42,내일,dlcj****,166.0,2.0,0.0
11,2021.02.16 17:42,96800에 물린 20대 초반 여대생입니... [5],cucu****,573.0,1.0,1.0


## 특정 주식 종목의 종목토론실 마지막 페이지 번호 호출 함수 생성

In [146]:
def get_last_page_no(item_code,headers = None):
    """
    특정 주식 종목의 종목토론실 마지막 페이지 번호 가져오기
    """
    url = f"https://finance.naver.com/item/board.nhn?code={item_code}&page=1"
    response = requests.get(url,headers = headers)
    
    html = bs(response.text)
    last_page_no = html.select(".pgRR > a")[0]["href"].split("=")[-1]
    return int(last_page_no)


# 반복문을 통해 전체 페이지 데이터 수집

In [187]:
item_code = "005930"
page_no = 1
headers = {'user-agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.150 Safari/537.36'}

results = []

#마지막 페이지 번호가 40000번대라 임시로 마지막 페이지 번호 지정
# last_pg_no = get_last_page_no(item_code = item_code, headers = headers)
last_pg_no = 20
while True:    
    result = get_page_data(item_code = item_code, page_no = page_no, headers = headers)
    results.append(result)
    notice = f"{page_no} page 수집 완료 [{round(page_no/last_pg_no*100,2)}%]"
    print(notice)
    page_no +=1
    time.sleep(np.random.randint(low = 1, high = 20)/10)
    if page_no > last_pg_no:
        break
    

1 page 수집 완료 [5.0%]
2 page 수집 완료 [10.0%]
3 page 수집 완료 [15.0%]
4 page 수집 완료 [20.0%]
5 page 수집 완료 [25.0%]
6 page 수집 완료 [30.0%]
7 page 수집 완료 [35.0%]
8 page 수집 완료 [40.0%]
9 page 수집 완료 [45.0%]
10 page 수집 완료 [50.0%]
11 page 수집 완료 [55.0%]
12 page 수집 완료 [60.0%]
13 page 수집 완료 [65.0%]
14 page 수집 완료 [70.0%]
15 page 수집 완료 [75.0%]
16 page 수집 완료 [80.0%]
17 page 수집 완료 [85.0%]
18 page 수집 완료 [90.0%]
19 page 수집 완료 [95.0%]
20 page 수집 완료 [100.0%]


# 수집한 데이터 합치기

In [192]:
results = pd.concat(results)
results.head()

,날짜,제목,글쓴이,조회,공감,비공감
1,2021.02.16 18:43,죄는 삼성회장님이 했는데,kyj3****,4.0,0.0,0.0
2,2021.02.16 18:43,삼성폭락?,7598****,6.0,0.0,0.0
3,2021.02.16 18:43,사상 최고로 개미들돈이 들어와 주가 끓어... [1],pres****,13.0,0.0,0.0
4,2021.02.16 18:42,저녁만 되면 설치는 간첩단 넘들.,peop****,11.0,2.0,2.0
5,2021.02.16 18:44,개보수가 김정은 지령받고 문재인 욕한다,tjdr****,1.0,0.0,0.0


# 종목코드 컬럼 생성

In [193]:
results["종목코드"] = item_code
results.head(2)

,날짜,제목,글쓴이,조회,공감,비공감,종목코드
1,2021.02.16 18:43,죄는 삼성회장님이 했는데,kyj3****,4.0,0.0,0.0,005930
2,2021.02.16 18:43,삼성폭락?,7598****,6.0,0.0,0.0,005930
